# Turning An Equation Into An Object
#### A description of the series of steps I used to successfully turn a set of parametric equations into a physical object, using Apple Grapher, iPython, Jupyter Notebook, NumPy, Mayavi and Rhinoceros
Chances are good that if your education proceeded as far as high school, at some point in your math curriculum you were exposed to the concept of equations, and that you learned how to draw graphs of some simple equations.  If so, you're familiar with the notion that a mathematical thing can have a shape.  You may also have seen examples of this notion applied to three-dimensional shapes.  Given the much-hyped advent of affordable 3D printing, you might wonder if it is possible for someone who knows a little bit of math, and a little bit about computers, to come up with a mathematical thing with an interesting shape, and somehow turn that shape into a physical object.  

I wondered that, and I tried to do it, and I succeeded.  I found a lot of information on the Internet, but nothing that described this process end-to-end in a way that worked for me, so I synthesized various bits and pieces I found in various places, and I went through a lot of trial and error and a lot of dead ends and pitfalls.  The whole time, I knew that it might be the case that what I was trying to accomplish was not possible at all, or at the very least not feasible for one non-expert to attempt as a hobby project.  I provide this writeup so that someone else who has wondered the same thing can know ahead of time that the answer is *yes, it is possible*.  

This is not a comprehensive set of instructions.  It does not describe in a general way how anyone should go about accomplishing a particular task, in that 1.) it does not describe how to customize the procedure to your own purposes, 2.) it does not describe how to adapt the procedure to varying prerequisites, 3.) it does not include extensive advice on how to recover from errors and 4.) it isn't even necessarily the best way I could have done it.  It is a record of a particular workflow that I know to have worked at least once.  I provide it on the theory that, when one is working with a complex heterogeneous set of interacting tools and parameters, it is often easier to start by replicating a workflow that is known to have worked, and then modifying it to your own purposes, debugging one modification at a time, than it is to develop an entire toolchain on your own, ex nihilo.  

#### Prerequisites
Here are the things I started with when I embarked on this project:  
* A MacBook Pro from 2013, running OS X Yosemite
* Some experience scripting with [Python][1]
* Fuzzy memories of high school algebra and trigonometry
* An account at [Shapeways][2]

You can very probably do all the same things on a Windows or Linux PC, and using another 3D printing service or even your own 3D printer, but these are the things I used.  

 [1]: http://www.python.org/
 [2]: http://www.shapeways.com/

#### Noodling In Grapher
Recent versions of OS X come with a program called Grapher.  I have the impression that Grapher has a wide range of impressive capabilities, but I use it as an equation editor and a graphing calculator.  It makes an excellent environment for experimenting with various mathematical things and seeing what they look like.  There are other more elaborate tools out there that will do the same thing, but this one is quick and easy to learn and comes free with every Mac.  

Grapher's equation editor lets you make equations that look the same as what you'd write on paper, with horizontal lines for division, superscripts for exponents, etc.  It may be unfamiliar, it's not like a word processor or a spreadsheet, but with a little practice you'll be able to write basic equations quickly and easily.  

Open Grapher.  An easy way is to invoke Spotlight with cmd-space, type "grapher" and hit enter.  You should see a "New Graph" dialog.  Start with a simple 2D graph.  With "2D Graph" selected at the top, and "Default" selected in the left pane, click "Choose".  You'll see a graph window, with the beginnings of an equation already set up for you:  "y=".  Type "x" then type the "^" character.  The "^" will create a box for a superscript.  Type "2" and the 2 should appear in the superscript box.  Hit enter, and Grapher shows a parabola, the shape of $y=x^{2}$.  

Noodle around for a bit.  Click the plus sign in the lower left corner, and select "New Equation".  Add an equation and see what it looks like.  If you want to add a horizontal line for a fraction, type "/".  With experimentation and Googling, you'll be able to figure out lots of similar shortcuts.  In the Git repository with this notebook, there's a Grapher file called "basic_curves.gcx".  It contains a handful of simple equations.  I made it for myself as a kind of catalog;  if I'm playing with a shape and I want to give it a curvature that's symmetric about y=x, I can see at a glance that $y=\frac{1}{x}$ is a good place to start.  You may want to make a similar catalog for yourself with simple features you want to use as aspects of more complex shapes.  

...



From this ![the equation][equation] to this ![the figure][figure] to this ![the printed object][object]

[equation]: images/ruffle_equation.png
[figure]: images/wrapped_ruffle.png
[object]: images/printed.png

#### Translating Equations to NumPy

In [1]:
%gui wx

In [2]:
from numpy import pi, sin, cos, mgrid
from mayavi import mlab
# You'll get a deprecation warning.  It's safe to ignore it.

/usr/lib/python2.7/dist-packages/traits/etsconfig/etsconfig.py:373: UserWarning: Environment variable "HOME" not set, setting home directory to /tmp
  (environment_variable, parent_directory))


In [3]:
def wrap(x, y, z):
    x1 = x * cos(y)
    y1 = x * sin(y)
    z1 = z
    return (x1, y1, z1)

This equation:  
$$
\left[ \begin{array}{c} x \\ y \\ z \end{array} \right]=\left[ \begin{array}{c} \frac{\sin \left( u\; \cos \left( 6v \right) \right)}{\cos \left( 6v \right)} \\ v\; +\; \left( \frac{u\; \sin \left( 12v \right)}{2^{3}} \right) \\ \frac{1-\cos \left( u\; \cos \left( 6v \right) \right)}{\cos \left( 6v \right)} \end{array} \right]
$$
can be transdlated into the Python function below:  

In [4]:
def ruffle(u, v):
    x = sin(u * cos(v*6)) / cos(v*6)
    y = v + (u * sin(12*v)/2**3)
    z = (1 - cos(u * cos(v*6))) / cos(v*6)
    return (x, y, z)

In [5]:
inner = pi/2**5
outer = pi/2

du, dv = 0.05*pi, 0.01*pi
u, v = mgrid[inner:outer+du:du, 0:2*pi+dv:dv]

#### Making A `mayavi.mlab` Mesh

In [6]:
mlab.mesh(*wrap(*ruffle(u, v)))

#### Exporting A `.obj` File From Mayavi

In [7]:
mlab.savefig("/data/notebooks/wrapped_ruffle.obj")

#### Offsetting The Mesh And Exporting STL In Blender

In [8]:
import subprocess
subprocess.call(["blender", "--background", "--python", "/data/examples/solidify.py"])

0

#### Uploading The `.stl` File To Shapeways For Printing